In [51]:
from sqlalchemy import create_engine, inspect
import creds
import pandas as pd
#For ML, we need:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [52]:
host = creds.host
password = creds.password
port = creds.port
user = creds.user
database = creds.database
api_type = creds.api_type
engine = create_engine(f'postgresql+{api_type}://{user}:{password}@{host}:{port}/{database}')
# engine.connect()

In [53]:
df = pd.read_sql_table('products', engine)
df.drop(['create_time', 'page_id'], axis=1, inplace=True)

In [68]:
### Cleaning up the price column in the dataframe.
# Removes all rows which have "N/A" in the price column.
df = df[df['price'] != 'N/A']
# df = df[df['product_name'] != 'N/A']
df['price'] = df['price'].str.strip('£')
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].astype('float64')

In [69]:
cvec = CountVectorizer()
X = df['product_name']
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [70]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train = pd.DataFrame(cvec.transform(X_train).todense(),columns=cvec.get_feature_names_out())

In [71]:
df_test = pd.DataFrame(cvec.transform(X_test).todense(),columns=cvec.get_feature_names_out())
print(df_train.shape)
print(y_train.shape)
print(df_test.shape)
print(y_test.shape)

(4794, 8280)
(4794,)
(2362, 8280)
(2362,)


In [66]:
df.head(10)

,id,product_name,category,product_description,price,location
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire"
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland"
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire"
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset"
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester"
6,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,£9.00,"Newent, Gloucestershire"
7,d2244a7b-aef4-4495-aff7-0c55bf512d6e,💥💥White Wardrobe💥💥Brand new Wardrobe For sale ...,Home & Garden / Beds & Bedroom Furniture / War...,🌈🌈Call Dustin 0208-0049437\r🌈🌈Same Day Or any...,£419.00,"Huddersfield, West Yorkshire"
8,485edc04-7bbc-430a-9430-f05a77f5a917,"Letter cage | in Newent, Gloucestershire | Gum...",Home & Garden / Other Household Goods,Selling for family member brand new too big fo...,£10.00,"Newent, Gloucestershire"
9,7e538445-c75e-4802-8dc5-a9083a802278,"Double Pine Bed Frame | in Lytham St Annes, La...",Home & Garden / Beds & Bedroom Furniture / Dou...,Double Pine bed frame good condition. Reasonab...,£50.00,"Lytham St Annes, Lancashire"
10,1a64843d-72f3-4141-946f-fa9b978fe755,Solid pine dining table and chairs | in West K...,"Home & Garden / Dining, Living Room Furniture ...",Distressed solid pine table and 4 chairs In ex...,£200.00,"West Kilbride, North Ayrshire"


In [67]:
lr = LogisticRegression()
lr.fit(df_train, y_train)
lr.score(df_test, y_test)

/home/oliver/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.12320067739204064